What are the possible base models? 


Model selectors 
- One for selecting either long term or short term 
- One for selecting which model prediction to trust
- One for online asset selection 

Mean Reversion 
- [x] OLMAR 
- [x] PAMR
- [x] WMAMR
- RPRT
- RMR 
- TCO 

Follow the Leader 
- [x] ONS 
- [x] Exponential Gradient
- BestSoFar 

Benchmark
- [x] Uniform CRP 
- Dynamic CRP 

Pattern Matching 
- CORN-K
- CORN-U  
- BNN 

Price Prediction 
- Informer based portfolio selection 
- Bidirectional-LSTM based portfolio selection 
- Online-ARIMA based portfolio selection  

Downmarket Defense
- [x] Cash Agent 

We can play with 10 different crypto currencies. Out of them, we may form 1024 different subsets. Using this, we may compare the returns of the subsets to decide which asset we need to select. 

Also, all models are designed to have the member functions 'update' and 'predict'. 
The update function calculates the next portfolio, and the predict function uses the update function to return the next portfolio.

In [259]:
import numpy as np 
import pandas as pd 
import random 
import os 

from cvxopt import matrix, solvers
from tqdm import tqdm
import pyupbit
import ccxt
import time
import datetime
import scipy.optimize as optimize
from tensorflow.keras.models import *
from tensorflow.keras.layers import *  
from tensorflow.keras.callbacks import *
import matplotlib.pyplot as plt
import json
import tensorflow as tf 
import torch
from numpy.linalg import inv 
from numpy import diag, log, sqrt, trace
import scipy.stats 

# Define Utility Functions

In [260]:
# Simplex Projection
def simplex_proj(y):
    """ Projection of y onto simplex. """
    m = len(y)
    bget = False

    s = sorted(y, reverse=True)
    tmpsum = 0.

    for ii in range(m-1):
        tmpsum = tmpsum + s[ii]
        tmax = (tmpsum - 1) / (ii + 1);
        if tmax >= s[ii+1]:
            bget = True
            break

    if not bget:
        tmax = (tmpsum + s[m-1] -1)/m

    y = np.asarray(y)

    return np.maximum(y-tmax,0.)

In [261]:
# w is the window size
# r is the rebalancing period in hours - gets every rth hour prices       
# obtains predicted return using the mean reversion method 
def mean_reversion_predicted_returns(ticker, w=5, r=10):
    df = pyupbit.get_ohlcv(ticker, interval="minute60")
    close = df['close'].values
    close = close[-1::-r] # get every 10th
    ma_prediction = 0
    for i in range(w):
        ma_prediction += close[i]
    ma_prediction *= (1/close[0]) * (1/w)
    return ma_prediction

In [262]:
# r is the rebalancing period in hours
def get_current_returns(ticker, r=10):
    df = pyupbit.get_ohlcv(ticker, interval="minute60")
    close = df['close'].values
    close = close[-1::-r] # get every 10th 
    returns = close[0] / close[1] # r_t(j) = p_t(j) / p_{t-1}(j)  
    return returns 

# Define Base Models

In [263]:
class PAMR: 
    def __init__(self, epsilon, C, variant, tickers):
        self.eps = epsilon 
        self.C = C 
        self.variant = variant 
        self.tickers = tickers
        
    def update(self, current_portfolio, x): 
        x_mean = np.mean(x) 
        le = max(0.0, np.dot(current_portfolio,x) - self.eps)
        if self.variant == 0: 
            lam = le / np.linalg.norm(x - x_mean) ** 2 
        elif self.variant == 1: 
            lam = min(self.C, le / np.linalg.norm(x - x_mean) ** 2)
        elif self.variant == 2: 
            lam = le / (np.linalg.norm(x - x_mean) ** 2 + 0.5 / self.C)  
        lam = min(100000, lam) 
        new_portfolio = current_portfolio - lam * (x - x_mean) 
        return simplex_proj(new_portfolio)
    
    def predict(self,current_portfolio,r,rtypes):
        if rtypes == "1" or rtypes == "10": 
            x = [] 
            for ticker in tqdm(self.tickers, position = 0, leave = True):
                while True: 
                    try: 
                        df = pyupbit.get_ohlcv(ticker, interval='minute60') 
                        df = df['open'].values 
                        df = df[r-1::r] # get every rth close prices 
                        ret = df[-1] / df[-2] 
                        x.append(ret) 
                        break 
                    except Exception as e: 
                        print(e)
                        continue 
            new_portfolio = self.update(current_portfolio, x)
            return new_portfolio  
        elif rtypes == "0.5":  
            x = [] 
            for ticker in tqdm(self.tickers, position = 0, leave = True):
                while True: 
                    try: 
                        df = pyupbit.get_ohlcv(ticker, interval='minute30') 
                        df = df['open'].values 
                        ret = df[-1] / df[-2] 
                        x.append(ret) 
                        break 
                    except Exception as e: 
                        print(e)
                        continue 
            new_portfolio = self.update(current_portfolio, x)
            return new_portfolio 
        elif rtypes == "4": 
            x = [] 
            for ticker in tqdm(self.tickers, position = 0, leave = True):
                while True: 
                    try: 
                        df = pyupbit.get_ohlcv(ticker, interval='minute240') 
                        df = df['open'].values 
                        ret = df[-1] / df[-2] 
                        x.append(ret) 
                        break 
                    except Exception as e: 
                        print(e)
                        continue 
            new_portfolio = self.update(current_portfolio, x)
            return new_portfolio 


In [264]:
class OLVWMA: 
    def __init__(self, epsilon, window_size, tickers):
        self.epsilon = epsilon 
        self.window_size = window_size 
        self.tickers = tickers 
        
    def update(self, current_portfolio, predicted_returns): 
        avg_x = np.sum([x*y for x,y in zip([1 for i in predicted_returns], predicted_returns)]) 
        avg_x *= 1.0 / float(len(predicted_returns))
        # numerator 
        temp_top = self.epsilon - sum([x*y for x,y in zip(current_portfolio, predicted_returns)]) 
        # denominator 
        temp_bottom1 = np.array(predicted_returns) 
        temp_bottom2 = np.array([avg_x for i in predicted_returns]) 
        dist = np.sum(np.power((temp_bottom1 - temp_bottom2), 2)) 
        lambda_1 = max(0.0, temp_top / dist) 
        # update portfolio 
        np_current_portfolio = np.array(current_portfolio) 
        new_portfolio = np_current_portfolio + lambda_1 * (temp_bottom1 - temp_bottom2) 
        new_portfolio = simplex_proj(new_portfolio) 
        return new_portfolio 
    
    def predict(self, current_portfolio, r, rtypes): 
        predicted_returns = [] 
        if rtypes == "1" or rtypes == "10":  
            for ticker in tqdm(self.tickers, position=0, leave=True):
                while True:
                    try: 
                        df = pyupbit.get_ohlcv(ticker, interval='minute60') 
                        close = df['open'].values 
                        vol = df['volume'].values 
                        close = close[r-1::r] 
                        vol_sum = [] 
                        for i in range(0,len(vol),r): 
                            vol_sum.append(np.sum(vol[i:i+r]))
                        vol_sum = np.asarray(vol_sum)
                        close = close[-self.window_size:] 
                        vol = vol_sum[-self.window_size:] 
                        top = 0 
                        bottom = 0 
                        for i in range(self.window_size): 
                            top += close[i] * vol[i] 
                            bottom += vol[i] 
                        vwma = top/bottom 
                        predicted_return = vwma / close[-1]  
                        predicted_returns.append(predicted_return)
                        break 
                    except Exception as e:
                        print(e) 
                        continue 
        elif rtypes == "0.5": 
             for ticker in tqdm(self.tickers, position=0, leave=True):
                while True:
                    try: 
                        df = pyupbit.get_ohlcv(ticker, interval='minute30') 
                        close = df['open'].values 
                        vol = df['volume'].values 
                        close = close[-self.window_size:] 
                        vol = vol[-self.window_size:] 
                        top = 0 
                        bottom = 0 
                        for i in range(self.window_size): 
                            top += close[i] * vol[i] 
                            bottom += vol[i] 
                        vwma = top/bottom 
                        predicted_return = vwma / close[-1]  
                        predicted_returns.append(predicted_return)
                        break 
                    except Exception as e:
                        print(e) 
                        continue  
        elif rtypes == "4": 
            for ticker in tqdm(self.tickers, position=0, leave=True):
                while True:
                    try: 
                        df = pyupbit.get_ohlcv(ticker, interval='minute240') 
                        close = df['open'].values 
                        vol = df['volume'].values 
                        close = close[-self.window_size:] 
                        vol = vol[-self.window_size:] 
                        top = 0 
                        bottom = 0 
                        for i in range(self.window_size): 
                            top += close[i] * vol[i] 
                            bottom += vol[i] 
                        vwma = top/bottom 
                        predicted_return = vwma / close[-1]  
                        predicted_returns.append(predicted_return)
                        break 
                    except Exception as e:
                        print(e) 
                        continue  

        new_portfolio = self.update(current_portfolio, predicted_returns) 
        return new_portfolio 
            

In [265]:
class RMR: 
    def __init__(self, window_size, eps, tau, tickers):  
        self.window_size = window_size 
        self.eps = eps 
        self.tau = tau  
        self.tickers = tickers 
        
    def norm(self, x):
        if isinstance(x, pd.Series):  
            axis = 0 
        else: 
            axis = 1 
        return np.sqrt((x**2).sum(axis=axis)) 
    
    def update(self, current_portfolio, predicted_returns): 
        avg_x = np.sum([x*y for x,y in zip([1 for i in predicted_returns], predicted_returns)])
        avg_x *= 1.0 / float(len(predicted_returns))
        # numerator
        temp_top = self.eps - sum([x*y for x,y in zip(current_portfolio, predicted_returns)])
        # denominator
        temp_bottom1 = np.array(predicted_returns)
        temp_bottom2 = np.array([avg_x for i in predicted_returns])
        dist = np.sum(np.power((temp_bottom1 - temp_bottom2), 2))
        lambda_1 = max(0.0, temp_top/dist)
        # update portfolio
        np_current_portfolio = np.array(current_portfolio)
        new_portfolio = np_current_portfolio + lambda_1 * (temp_bottom1 - temp_bottom2)
        new_portfolio = simplex_proj(new_portfolio.tolist())
        return new_portfolio        
    
    def predict(self, current_portfolio, r, rtypes):  
        prices = []  
        open_prices = [] 
        if rtypes == "1" or rtypes == "10":
            for ticker in tqdm(tickers, position=0, leave=True): 
                while True: 
                    try: 
                        df = pyupbit.get_ohlcv(ticker, interval='minute60') 
                        price = df['open'] 
                        price = price.iloc[r-1::r] 
                        price = price.iloc[-self.window_size:] 
                        prices.append(price)  
                        open_prices.append(price.values[-1])  
                        break 
                    except Exception as e: 
                        print(e)
                        continue 
        elif rtypes == "0.5": 
            for ticker in tqdm(tickers, position=0, leave=True):
                while True: 
                    try: 
                        df = pyupbit.get_ohlcv(ticker, interval='minute30') 
                        price = df['open'] 
                        price = price.iloc[-self.window_size:] 
                        prices.append(price)  
                        open_prices.append(price.values[-1]) 
                        break 
                    except Exception as e: 
                        print(e)
                        continue 
        elif rtypes == "4": 
            for ticker in tqdm(tickers, position=0, leave=True): 
                while True: 
                    try: 
                        df = pyupbit.get_ohlcv(ticker, interval='minute240') 
                        price = df['open'] 
                        price = price.iloc[-self.window_size:] 
                        prices.append(price)  
                        open_prices.append(price.values[-1])  
                        break 
                    except Exception as e: 
                        print(e)
                        continue 

        df_price = pd.DataFrame(prices).T 

        y = df_price.mean() 
        y_last = None 
        eps = 1e-10 # prevent nan or zero divisions
        while y_last is None or self.norm(y - y_last) / (self.norm(y_last)+eps) > self.tau: 
            y_last = y 
            d = self.norm(df_price - y) 
            y = df_price.div(d+eps, axis=0).sum() / ((1.0 / (d+eps)).sum() + eps)

        y = np.asarray(y)
        open_prices = np.asarray(open_prices) 
        predicted_returns = y / (open_prices + eps) 

        new_portfolio = self.update(current_portfolio, predicted_returns) 
        return new_portfolio 

In [266]:
class OLMAR:
    def __init__(self, epsilon, window_size, tickers):
        self.epsilon = epsilon 
        self.window_size = window_size 
        self.tickers = tickers 
        
    def update(self, current_portfolio, predicted_returns): 
        avg_x = np.sum([x*y for x,y in zip([1 for i in predicted_returns], predicted_returns)])
        avg_x *= 1.0 / float(len(predicted_returns))
        # numerator
        temp_top = self.epsilon - sum([x*y for x,y in zip(current_portfolio, predicted_returns)])
        # denominator
        temp_bottom1 = np.array(predicted_returns)
        temp_bottom2 = np.array([avg_x for i in predicted_returns])
        dist = np.sum(np.power((temp_bottom1 - temp_bottom2), 2))
        lambda_1 = max(0.0, temp_top/dist)
        # update portfolio
        np_current_portfolio = np.array(current_portfolio)
        new_portfolio = np_current_portfolio + lambda_1 * (temp_bottom1 - temp_bottom2)
        new_portfolio = simplex_proj(new_portfolio.tolist())
        return new_portfolio 
    
    def predict(self, current_portfolio, r, rtypes): 
        predicted_returns = [] 
        if rtypes == "1" or rtypes == "10":  
            for ticker in tqdm(self.tickers, position = 0, leave=True): 
                while True:  
                    try: 
                        df = pyupbit.get_ohlcv(ticker, interval='minute60') 
                        close = df['open'].values 
                        close = close[r-1::r] 
                        close = close[-self.window_size:] 
                        ma = np.sum(close) * (1/close[-1]) * (1/self.window_size)  
                        predicted_returns.append(ma) 
                        break 
                    except Exception as e: 
                        print(e) 
                        continue 
                        
        elif rtypes == "0.5":  
            for ticker in tqdm(self.tickers, position = 0, leave=True): 
                while True:  
                    try: 
                        df = pyupbit.get_ohlcv(ticker, interval='minute30') 
                        close = df['open'].values 
                        close = close[-self.window_size:] 
                        ma = np.sum(close) * (1/close[-1]) * (1/self.window_size)  
                        predicted_returns.append(ma) 
                        break 
                    except Exception as e: 
                        print(e) 
                        continue  
                        
        elif rtypes == "4":  
            for ticker in tqdm(self.tickers, position = 0, leave=True): 
                while True:  
                    try: 
                        df = pyupbit.get_ohlcv(ticker, interval='minute240') 
                        close = df['open'].values 
                        close = close[-self.window_size:] 
                        ma = np.sum(close) * (1/close[-1]) * (1/self.window_size)  
                        predicted_returns.append(ma) 
                        break 
                    except Exception as e: 
                        print(e) 
                        continue  


        new_portfolio = self.update(current_portfolio, predicted_returns) 
        return new_portfolio 
    

In [267]:
class WMAMR: 
    def __init__(self, epsilon, C, variant, window_size, tickers): 
        self.eps = epsilon
        self.C = C 
        self.variant = variant 
        self.window_size = window_size 
        self.tickers = tickers 
    
    def update(self, current_portfolio, x): 
        x_mean = np.mean(x) 
        le = max(0.0, np.dot(current_portfolio,x) - self.eps)
        if self.variant == 0: 
            lam = le / np.linalg.norm(x - x_mean) ** 2 
        elif self.variant == 1: 
            lam = min(self.C, le / np.linalg.norm(x - x_mean) ** 2)
        elif self.variant == 2: 
            lam = le / (np.linalg.norm(x - x_mean) ** 2 + 0.5 / self.C)  
        lam = min(100000, lam) 
        new_portfolio = current_portfolio - lam * (x - x_mean) 
        return simplex_proj(new_portfolio)

    def predict(self, current_portfolio, r, rtypes): 
        xx = [] 
        if rtypes == "1" or rtypes == "10": 
            for ticker in tqdm(self.tickers, position = 0, leave=True):
                while True: 
                    try: 
                        df = pyupbit.get_ohlcv(ticker, interval='minute60') 
                        df = df['open'].values 
                        df = df[r-1::r] # get every rth close prices  
                        simple_mean = np.mean(df[-self.window_size:]) 
                        xx.append(simple_mean/df[-1])
                        break 
                    except Exception as e:
                        print(e) 
                        continue 
        elif rtypes == "0.5":  
            for ticker in tqdm(self.tickers, position = 0, leave=True):
                while True: 
                    try: 
                        df = pyupbit.get_ohlcv(ticker, interval='minute30') 
                        df = df['open'].values 
                        simple_mean = np.mean(df[-self.window_size:]) 
                        xx.append(simple_mean/df[-1])
                        break 
                    except Exception as e:
                        print(e) 
                        continue  
                        
        elif rtypes == "4": 
            for ticker in tqdm(self.tickers, position = 0, leave=True):
                while True: 
                    try: 
                        df = pyupbit.get_ohlcv(ticker, interval='minute240') 
                        df = df['open'].values 
                        simple_mean = np.mean(df[-self.window_size:]) 
                        xx.append(simple_mean/df[-1])
                        break 
                    except Exception as e:
                        print(e) 
                        continue 

        new_portfolio = self.update(current_portfolio, xx) 
        return new_portfolio

In [268]:
class RPRT: 
    def __init__(self, window_size, eps, theta, tickers): 
        self.window_size = window_size
        self.eps = eps 
        self.theta = theta 
        self.phi = np.ones(len(tickers))
        self.tickers = tickers 
    
    def update(self, current_portfolio, phi_pred, D_pred): 
        phi_pred_mean = np.mean(phi_pred) 
        if np.linalg.norm(phi_pred - phi_pred_mean) ** 2 == 0: 
            lam = 0 
        else: 
            lam = max(
                0.0,
                (self.eps - np.dot(current_portfolio, phi_pred))
                / np.linalg.norm(phi_pred - phi_pred_mean) ** 2, 
            )
            
        # update portfolio 
        new_portfolio = current_portfolio + lam * np.dot(D_pred, (phi_pred - phi_pred_mean)) 
        return simplex_proj(new_portfolio) 
    
    def predict(self, current_portfolio, r, rtypes): 
        x_pred = [] # predict price on the next day 
        last_price_relative = [] 
        
        if rtypes == "1" or rtypes == "10":  
            for ticker in tqdm(self.tickers, position=0, leave=True): 
                while True:
                    try: 
                        df = pyupbit.get_ohlcv(ticker, interval='minute60') 
                        df = df['open'].values 
                        df = df[r-1::r] # get every rth close prices  
                        df = df[-self.window_size:] 
                        ma = np.mean(df) 
                        x_pred.append(ma)  
                        last_price_relative.append(df[-1])
                        break 
                    except Exception as e: 
                        print(e) 
                        continue 
        elif rtypes == "0.5": 
            for ticker in tqdm(self.tickers, position=0, leave=True): 
                while True:
                    try: 
                        df = pyupbit.get_ohlcv(ticker, interval='minute30') 
                        df = df['open'].values 
                        df = df[-self.window_size:] 
                        ma = np.mean(df) 
                        x_pred.append(ma)  
                        last_price_relative.append(df[-1])
                        break 
                    except Exception as e: 
                        print(e) 
                        continue 
        elif rtypes == "4": 
            for ticker in tqdm(self.tickers, position=0, leave=True): 
                while True:
                    try: 
                        df = pyupbit.get_ohlcv(ticker, interval='minute240') 
                        df = df['open'].values 
                        df = df[-self.window_size:] 
                        ma = np.mean(df) 
                        x_pred.append(ma)  
                        last_price_relative.append(df[-1])
                        break 
                    except Exception as e: 
                        print(e) 
                        continue 
    

        D_pred = np.diag(np.array(x_pred)) 
        last_phi = self.phi
        last_price_relative = np.asarray(last_price_relative) 
        
        gamma_pred = (
            self.theta
            * last_price_relative
            / (self.theta * last_price_relative + last_phi) 
        )
        phi_pred = gamma_pred + np.multiply(
            1 - gamma_pred, np.divide(last_phi, last_price_relative)
        )
        self.phi = phi_pred 
        # update weights
        new_portfolio = self.update(current_portfolio, phi_pred=phi_pred, D_pred=D_pred) 
        return new_portfolio 

In [269]:
class ONS: 
    def __init__(self, delta, beta, eta, assets, tickers): 
        self.delta = delta 
        self.beta = beta
        self.eta = eta 
        self.assets = assets  
        self.tickers = tickers 
        self.A = np.mat(np.eye(self.assets)) 
        self.b = np.mat(np.zeros(self.assets)).T 
        
    def projection_in_norm(self,x,M): 
        """Projection of x to simplex indiced by matrix M. Uses quadratic programming."""
        m = M.shape[0]
        P = matrix(2 * M)
        q = matrix(-2 * M * x)
        G = matrix(-np.eye(m))
        h = matrix(np.zeros((m, 1)))
        A = matrix(np.ones((1, m)))
        b = matrix(1.0)
        solvers.options["show_progress"] = False
        sol = solvers.qp(P, q, G, h, A, b)
        return np.squeeze(sol["x"]) 
    
    def update(self,current_portfolio, current_returns):
        # calculate gradient 
        grad = np.mat(current_returns / np.dot(current_portfolio, current_returns)).T
        # update A 
        self.A += grad * grad.T
        # update b 
        self.b += (1 + 1.0 / self.beta) * grad 
        # projection of p induced by norm A 
        pp = self.projection_in_norm(self.delta * self.A.I * self.b, self.A)
        return pp * (1 - self.eta) + np.ones(len(current_returns)) / float(len(current_returns)) * self.eta
    
    def predict(self, current_portfolio,r,rtypes):  
        x = [] 
        if rtypes == "1" or rtypes == "10":  
            for ticker in tqdm(tickers, position = 0, leave = True):
                while True: 
                    try: 
                        df = pyupbit.get_ohlcv(ticker, interval='minute60') 
                        close = df['open'].values 
                        close = close[r-1::r]  
                        returns = close[-1] / close[-2]  
                        x.append(returns)
                        break 
                    except Exception as e: 
                        print(e) 
                        continue  
        elif rtypes == "0.5": 
            for ticker in tqdm(tickers, position = 0, leave = True):
                while True: 
                    try: 
                        df = pyupbit.get_ohlcv(ticker, interval='minute30') 
                        close = df['open'].values 
                        returns = close[-1] / close[-2]  
                        x.append(returns)
                        break 
                    except Exception as e: 
                        print(e) 
                        continue 
        elif rtypes == "4": 
            for ticker in tqdm(tickers, position = 0, leave = True):
                while True: 
                    try: 
                        df = pyupbit.get_ohlcv(ticker, interval='minute240') 
                        close = df['open'].values 
                        returns = close[-1] / close[-2]  
                        x.append(returns)
                        break 
                    except Exception as e: 
                        print(e) 
                        continue  


        new_portfolio = self.update(current_portfolio, x) 
        return new_portfolio 


In [270]:
class EXPONENTIAL_GRADIENT: 
    def __init__(self, eta, tickers): 
        self.eta = eta 
        self.tickers = tickers 
        
    def update(self, current_portfolio, current_returns): 
        current_portfolio = np.asarray(current_portfolio) 
        current_returns = np.asarray(current_returns)
        new_portfolio = current_portfolio * np.exp(self.eta * current_returns / sum(current_returns * current_portfolio))
        return new_portfolio / sum(new_portfolio) 
    
    def predict(self, current_portfolio, r, rtypes):  
        x = [] 
        
        if rtypes == "1" or rtypes == "10":  
            for ticker in tqdm(tickers, position = 0, leave = True):
                while True:
                    try: 
                        df = pyupbit.get_ohlcv(ticker, interval='minute60') 
                        close = df['open'].values 
                        close = close[r-1::r]  
                        returns = close[-1] / close[-2]  
                        x.append(returns) 
                        break 
                    except Exception as e: 
                        print(e) 
                        continue  
        elif rtypes == "0.5": 
            for ticker in tqdm(tickers, position = 0, leave = True):
                while True:
                    try: 
                        df = pyupbit.get_ohlcv(ticker, interval='minute30') 
                        close = df['open'].values 
                        returns = close[-1] / close[-2]  
                        x.append(returns) 
                        break 
                    except Exception as e: 
                        print(e) 
                        continue   
        elif rtypes == "4": 
            for ticker in tqdm(tickers, position = 0, leave = True):
                while True:
                    try: 
                        df = pyupbit.get_ohlcv(ticker, interval='minute240') 
                        close = df['open'].values 
                        returns = close[-1] / close[-2]  
                        x.append(returns) 
                        break 
                    except Exception as e: 
                        print(e) 
                        continue  


        new_portfolio = self.update(current_portfolio, x) 
        return new_portfolio 

In [271]:
class CWMR: 
    def __init__(self, eps, confidence, tickers): 
        self.eps = eps 
        self.theta = scipy.stats.norm.ppf(confidence)
        self.tickers = tickers  
        self.assets = len(tickers)
        self.sigma = np.matrix(np.eye(self.assets) / self.assets ** 2) 
        
    def update(self, x, x_upper, mu, sigma, M, V, theta, eps): 
        foo = (
            V - x_upper * x.T * np.sum(sigma, axis=1)
        ) / M ** 2 + V * theta ** 2 / 2.0
        a = foo ** 2 - V ** 2 * theta ** 4 / 4
        b = 2 * (eps - log(M)) * foo
        c = (eps - log(M)) ** 2 - V * theta ** 2

        a, b, c = a[0, 0], b[0, 0], c[0, 0]

        lam = max(
            0,
            (-b + sqrt(b ** 2 - 4 * a * c)) / (2.0 * a),
            (-b - sqrt(b ** 2 - 4 * a * c)) / (2.0 * a),
        )
        # bound it due to numerical problems
        lam = min(lam, 1e7)

        # update mu and sigma
        U_sqroot = 0.5 * (
            -lam * theta * V + sqrt(lam ** 2 * theta ** 2 * V ** 2 + 4 * V)
        )
        mu = mu - lam * sigma * (x - x_upper) / M
        sigma = inv(inv(sigma) + theta * lam / U_sqroot * diag(x) ** 2)
        return mu, sigma 
        
    def predict(self, current_portfolio, r, rtypes):  
        x = [] 
        
        if rtypes == "1" or rtypes == "10":  
            for ticker in tqdm(self.tickers, position = 0, leave=True):
                while True: 
                    try: 
                        df = pyupbit.get_ohlcv(ticker, interval='minute60') 
                        df = df['open'].values 
                        df = df[r-1::r] # get every rth close prices  
                        x.append(df[-1] / df[-2])  
                        break 
                    except Exception as e:
                        print(e) 
                        continue  
        elif rtypes == "0.5": 
            for ticker in tqdm(self.tickers, position = 0, leave=True):
                while True: 
                    try: 
                        df = pyupbit.get_ohlcv(ticker, interval='minute30') 
                        df = df['open'].values 
                        x.append(df[-1] / df[-2])  
                        break 
                    except Exception as e:
                        print(e) 
                        continue   
        elif rtypes == "4":  
            for ticker in tqdm(self.tickers, position = 0, leave=True):
                while True: 
                    try: 
                        df = pyupbit.get_ohlcv(ticker, interval='minute240') 
                        df = df['open'].values 
                        x.append(df[-1] / df[-2])  
                        break 
                    except Exception as e:
                        print(e) 
                        continue   

        m = len(x) 
        mu = np.matrix(current_portfolio).T 
        sigma = self.sigma 
        theta = self.theta 
        eps = self.eps 
        x = np.matrix(x).T 
        M = mu.T * x 
        V = x.T * sigma * x
        x_upper = sum(diag(sigma) * x) / trace(sigma) 
        mu, sigma = self.update(x, x_upper, mu, sigma, M, V, theta, eps)
        
        mu = simplex_proj(mu) # new portfolio 
        sigma = sigma / (m ** 2 * trace(sigma))
        self.sigma = sigma 
        return np.array(mu.T)[0]

In [272]:
'''
transactions cost optimization: single period mean reverting method 
'''  
class TCO1: 
    def __init__(self, trx_fee_pct, types, eta, tickers): 
        self.trx_fee_pct = trx_fee_pct 
        self.eta = eta 
        self.types = types 
        self.tickers = tickers 
    
    def update(self, x, current_portfolio, x_pred): 
        lambd = 10 * self.trx_fee_pct
        # last price adjusted by weights
        updated_portfolio = np.multiply(current_portfolio, x) / np.dot(current_portfolio, x)
        # calculate variables 
        vt = x_pred / np.dot(updated_portfolio, x_pred)
        v_t_ = np.mean(vt)
        # update portfolio 
        b_1 = self.eta * (vt - np.dot(v_t_, 1))
        b_ = updated_portfolio + np.sign(b_1) * np.maximum(
            np.zeros(len(b_1)), np.abs(b_1) - lambd
        )
        proj = simplex_proj(b_)
        return proj
    
    def predict(self, current_portfolio, r, rtypes): 
        x_pred = [] 
        x = [] 
        if rtypes == "1" or rtypes == "10":  
            if self.types == "reverting": 
                for ticker in tqdm(self.tickers, position=0, leave=True): 
                    while True: 
                        try: 
                            df = pyupbit.get_ohlcv(ticker, interval='minute60') 
                            close = df['open'].values 
                            close = close[r-1::r]
                            x_pred.append(close[-2] / close[-1])
                            x.append(close[-1] / close[-2]) 
                            break 
                        except Exception as e: 
                            print(e) 
                            continue 
            elif self.types == "trend": 
                for ticker in tqdm(self.tickers, position=0, leave=True): 
                    while True: 
                        try: 
                            df = pyupbit.get_ohlcv(ticker, interval='minute60') 
                            close = df['open'].values 
                            close = close[r-1::r]
                            x_pred.append(close[-1] / close[-2])
                            x.append(close[-1] / close[-2]) 
                            break 
                        except Exception as e: 
                            print(e) 
                            continue 
                            
        elif rtypes == "0.5": 
            if self.types == "reverting": 
                for ticker in tqdm(self.tickers, position=0, leave=True): 
                    while True: 
                        try: 
                            df = pyupbit.get_ohlcv(ticker, interval='minute30') 
                            close = df['open'].values 
                            x_pred.append(close[-2] / close[-1])
                            x.append(close[-1] / close[-2]) 
                            break 
                        except Exception as e: 
                            print(e) 
                            continue 
            elif self.types == "trend": 
                for ticker in tqdm(self.tickers, position=0, leave=True): 
                    while True: 
                        try: 
                            df = pyupbit.get_ohlcv(ticker, interval='minute30') 
                            close = df['open'].values 
                            x_pred.append(close[-1] / close[-2])
                            x.append(close[-1] / close[-2]) 
                            break 
                        except Exception as e: 
                            print(e) 
                            continue 
        elif rtypes == "4": 
            if self.types == "reverting": 
                for ticker in tqdm(self.tickers, position=0, leave=True): 
                    while True: 
                        try: 
                            df = pyupbit.get_ohlcv(ticker, interval='minute240') 
                            close = df['open'].values 
                            x_pred.append(close[-2] / close[-1])
                            x.append(close[-1] / close[-2]) 
                            break 
                        except Exception as e: 
                            print(e) 
                            continue 
            elif self.types == "trend": 
                for ticker in tqdm(self.tickers, position=0, leave=True): 
                    while True: 
                        try: 
                            df = pyupbit.get_ohlcv(ticker, interval='minute240') 
                            close = df['open'].values 
                            x_pred.append(close[-1] / close[-2])
                            x.append(close[-1] / close[-2]) 
                            break 
                        except Exception as e: 
                            print(e) 
                            continue 


        
        new_portfolio = self.update(x, current_portfolio, x_pred)
        return new_portfolio 

In [273]:
'''
transactions cost optimization: predict using OLMAR style (multi period MA)
'''
class TCO2: 
    def __init__(self, trx_fee_pct, eta, window_size, tickers): 
        self.trx_fee_pct = trx_fee_pct 
        self.eta = eta 
        self.window_size = window_size 
        self.tickers = tickers 
    
    def update(self, x, current_portfolio, x_pred): 
        lambd = 10 * self.trx_fee_pct
        # last price adjusted by weights
        updated_portfolio = np.multiply(current_portfolio, x) / np.dot(current_portfolio, x)
        # calculate variables 
        vt = x_pred / np.dot(updated_portfolio, x_pred)
        v_t_ = np.mean(vt)
        # update portfolio 
        b_1 = self.eta * (vt - np.dot(v_t_, 1))
        b_ = updated_portfolio + np.sign(b_1) * np.maximum(
            np.zeros(len(b_1)), np.abs(b_1) - lambd
        )
        proj = simplex_proj(b_)
        return proj
    
    def predict(self, current_portfolio, r, rtypes): 
        x_pred = [] 
        x = [] 
        
        if rtypes == "1" or rtypes == "10": 
            for ticker in tqdm(self.tickers, position=0, leave=True): 
                while True: 
                    try: 
                        df = pyupbit.get_ohlcv(ticker, interval='minute60') 
                        close = df['open'].values 
                        close = close[r-1::r]
                        close = close[-self.window_size:] 
                        ma = 0 
                        for i in range(self.window_size): 
                            ma += close[i]
                        ma = (1/close[-1]) * (1/self.window_size) * ma 
                        x_pred.append(ma) 
                        x.append(close[-1] / close[-2]) 
                        break 
                    except Exception as e: 
                        print(e) 
                        continue 
        elif rtypes == "0.5": 
            for ticker in tqdm(self.tickers, position=0, leave=True): 
                while True: 
                    try: 
                        df = pyupbit.get_ohlcv(ticker, interval='minute30') 
                        close = df['open'].values 
                        close = close[-self.window_size:] 
                        ma = 0 
                        for i in range(self.window_size): 
                            ma += close[i]
                        ma = (1/close[-1]) * (1/self.window_size) * ma 
                        x_pred.append(ma) 
                        x.append(close[-1] / close[-2]) 
                        break 
                    except Exception as e: 
                        print(e) 
                        continue 
                        
        elif rtypes == "4":  
            for ticker in tqdm(self.tickers, position=0, leave=True): 
                while True: 
                    try: 
                        df = pyupbit.get_ohlcv(ticker, interval='minute240') 
                        close = df['open'].values 
                        close = close[-self.window_size:] 
                        ma = 0 
                        for i in range(self.window_size): 
                            ma += close[i]
                        ma = (1/close[-1]) * (1/self.window_size) * ma 
                        x_pred.append(ma) 
                        x.append(close[-1] / close[-2]) 
                        break 
                    except Exception as e: 
                        print(e) 
                        continue 


        
        new_portfolio = self.update(x, current_portfolio, x_pred)
        return new_portfolio 

In [274]:
class UNIFORM_CRP: 
    def __init__(self, assets): 
        self.assets = assets 
    def update(self): 
        pass 
    def predict(self, current_portfolio): 
        return [1/self.assets for i in range(self.assets)] 

In [275]:
class CASH_AGENT: 
    def __init__(self, assets):  
        self.assets = assets 
    def update(self): 
        pass 
    def predict(self, current_portfolio):  
        return [0 for i in range(self.assets)]

# Long Term + Short Term Implementation 

Long Term rebalances every 10 hours 
Short Term rebalances every 1 hour 

Base Models for now 
- ONS 
- EG
- OLMAR (w=5) 
- OLMAR (w=10)
- OLMAR (w=15) 
- OLMAR (w=20) 
- OLVWMA (w=5) 
- OLVWMA (w=10) 
- OLVWMA (w=15) 
- OLVWMA (w=20) 
- RMR (w=5) 
- RMR (w=10) 
- RMR (w=15) 
- RMR (w=20) 
- WMAMR (w=5)
- WMAMR (w=10)  
- WMAMR (w=15) 
- WMAMR (w=20) 
- RPRT (w=5) 
- RPRT (w=10)  
- RPRT (w=15) 
- RPRT (w=20) 
- CWMR
- TCO1 
- TCO2 (w=5) 
- TCO2 (w=10)  
- TCO2 (w=15) 
- TCO2 (w=20) 
- PAMR 
- Uniform CRP 
- Cash Agent 

Model Selectors for now 
- base model selector for long term agent 
- base model selector for short term agent 
- agent model selector for choosing long term or short term agent 

In [276]:
class Agent:
    def __init__(self, r, rtypes, tickers, wgma_window_size, gma_window_size, name, gamma): 
        self.r = r # rebalancing period 
        self.rtypes = rtypes # needed for upbit API to view prices 
        self.tickers = tickers 
        self.assets = len(self.tickers)
        self.steps = 0  
        self.iterations = 0 
        self.name = name 
        self.gamma = gamma 
        # model selector parameters 
        self.wgma_window_size = wgma_window_size  
        self.gma_window_size = gma_window_size  
        
        # base models 
        ##### Mean Reversion Types ##### 
        self.pamr = PAMR(epsilon=0.5, C=500, variant=0, tickers=self.tickers)
        self.olmar5 = OLMAR(epsilon=10, window_size=5, tickers=self.tickers) 
        self.olmar10 = OLMAR(epsilon=10, window_size=10, tickers=self.tickers) 
        self.olmar15 = OLMAR(epsilon=10, window_size=15, tickers=self.tickers)
        self.olmar20 = OLMAR(epsilon=10, window_size=20, tickers=self.tickers) 
        self.olvwma5 = OLVWMA(epsilon=10, window_size=5, tickers=self.tickers) 
        self.olvwma10 = OLVWMA(epsilon=10, window_size=10, tickers=self.tickers) 
        self.olvwma15 = OLVWMA(epsilon=10, window_size=15, tickers=self.tickers) 
        self.olvwma20 = OLVWMA(epsilon=10, window_size=20, tickers=self.tickers) 
        self.rmr5 = RMR(window_size=5,eps=10,tau=0.001,tickers=self.tickers)  
        self.rmr10 = RMR(window_size=10,eps=10, tau=0.001,tickers=self.tickers) 
        self.rmr15 = RMR(window_size=15,eps=10,tau=0.001,tickers=self.tickers) 
        self.rmr20 = RMR(window_size=20,eps=10,tau=0.001,tickers=self.tickers)
        self.wmamr5 = WMAMR(epsilon=0.5, C=500, variant=0, window_size=5, tickers=self.tickers)  
        self.wmamr10 = WMAMR(epsilon=0.5, C=500, variant=0, window_size=10, tickers=self.tickers)
        self.wmamr15 = WMAMR(epsilon=0.5, C=500, variant=0, window_size=15, tickers=self.tickers) 
        self.wmamr20 = WMAMR(epsilon=0.5, C=500, variant=0, window_size=20, tickers=self.tickers) 
        self.rprt5 = RPRT(window_size=5, eps=50, theta=0.8, tickers=self.tickers) 
        self.rprt10 = RPRT(window_size=10, eps=50, theta=0.8, tickers=self.tickers)
        self.rprt15 = RPRT(window_size=15, eps=50, theta=0.8, tickers=self.tickers) 
        self.rprt20 = RPRT(window_size=20, eps=50, theta=0.8, tickers=self.tickers) 
        self.cwmr = CWMR(eps=-0.5,confidence=0.95,tickers=self.tickers)
        self.tco1 = TCO1(trx_fee_pct=self.gamma, eta=10, types="reverting", tickers=self.tickers) 
        self.tco2_5 = TCO2(trx_fee_pct=self.gamma, eta=10, window_size=5, tickers=self.tickers)  
        self.tco2_10 = TCO2(trx_fee_pct=self.gamma, eta=10, window_size=10, tickers=self.tickers)
        self.tco2_15 = TCO2(trx_fee_pct=self.gamma, eta=10, window_size=15, tickers=self.tickers) 
        self.tco2_20 = TCO2(trx_fee_pct=self.gamma, eta=10, window_size=20, tickers=self.tickers) 
        ##### Momentum Types #####
        self.ons = ONS(delta=0.125, beta=1.0, eta=0.0, assets=self.assets, tickers=self.tickers) 
        self.eg = EXPONENTIAL_GRADIENT(eta=0.05, tickers=self.tickers) 
        ##### Benchmarks ##### 
        self.uniform_crp = UNIFORM_CRP(self.assets) 
        self.cash_agent = CASH_AGENT(self.assets) 
        
        # base models performances for GMA model selector
        self.pamr_history = [] 
        self.olmar5_history = [] 
        self.olmar10_history = [] 
        self.olmar15_history = [] 
        self.olmar20_history = [] 
        self.olvwma5_history = [] 
        self.olvwma10_history = [] 
        self.olvwma15_history = [] 
        self.olvwma20_history = []  
        self.rmr5_history = [] 
        self.rmr10_history = [] 
        self.rmr15_history = [] 
        self.rmr20_history = [] 
        self.ons_history = []
        self.eg_history = [] 
        self.uniform_crp_history = []  
        self.wmamr5_history = [] 
        self.wmamr10_history = [] 
        self.wmamr15_history = [] 
        self.wmamr20_history = [] 
        self.rprt5_history = []
        self.rprt10_history = []
        self.rprt15_history = [] 
        self.rprt20_history = [] 
        self.cwmr_history = [] 
        self.tco1_history = []
        self.tco2_5_history = [] 
        self.tco2_10_history = []
        self.tco2_15_history = [] 
        self.tco2_20_history = [] 
        # base model performances for WGMA model selector 
        self.wgma_pamr_history = []
        self.wgma_olmar5_history = []
        self.wgma_olmar10_history = [] 
        self.wgma_olmar15_history = [] 
        self.wgma_olmar20_history = [] 
        self.wgma_olvwma5_history = [] 
        self.wgma_olvwma10_history = [] 
        self.wgma_olvwma15_history = [] 
        self.wgma_olvwma20_history = [] 
        self.wgma_rmr5_history = [] 
        self.wgma_rmr10_history = [] 
        self.wgma_rmr15_history = [] 
        self.wgma_rmr20_history = [] 
        self.wgma_ons_history = []
        self.wgma_eg_history = [] 
        self.wgma_uniform_crp_history = [] 
        self.wgma_wmamr5_history = [] 
        self.wgma_wmamr10_history = [] 
        self.wgma_wmamr15_history = [] 
        self.wgma_wmamr20_history = [] 
        self.wgma_rprt5_history = [] 
        self.wgma_rprt10_history = [] 
        self.wgma_rprt15_history = [] 
        self.wgma_rprt20_history = []  
        self.wgma_cwmr_history = []  
        self.wgma_tco1_history = [] 
        self.wgma_tco2_5_history = [] 
        self.wgma_tco2_10_history = [] 
        self.wgma_tco2_15_history = [] 
        self.wgma_tco2_20_history = [] 
        
        # base models portfolios 
        self.pamr_portfolio = [1/self.assets for i in range(self.assets)]
        self.olmar5_portfolio = [1/self.assets for i in range(self.assets)] 
        self.olmar10_portfolio = [1/self.assets for i in range(self.assets)] 
        self.olmar15_portfolio = [1/self.assets for i in range(self.assets)] 
        self.olmar20_portfolio = [1/self.assets for i in range(self.assets)] 
        self.olvwma5_portfolio = [1/self.assets for i in range(self.assets)] 
        self.olvwma10_portfolio = [1/self.assets for i in range(self.assets)]
        self.olvwma15_portfolio = [1/self.assets for i in range(self.assets)]
        self.olvwma20_portfolio = [1/self.assets for i in range(self.assets)] 
        self.rmr5_portfolio = [1/self.assets for i in range(self.assets)] 
        self.rmr10_portfolio = [1/self.assets for i in range(self.assets)] 
        self.rmr15_portfolio = [1/self.assets for i in range(self.assets)] 
        self.rmr20_portfolio = [1/self.assets for i in range(self.assets)] 
        self.ons_portfolio = [1/self.assets for i in range(self.assets)]
        self.eg_portfolio = [1/self.assets for i in range(self.assets)] 
        self.uniform_crp_portfolio = [1/self.assets for i in range(self.assets)] 
        self.wmamr5_portfolio = [1/self.assets for i in range(self.assets)] 
        self.wmamr10_portfolio = [1/self.assets for i in range(self.assets)] 
        self.wmamr15_portfolio = [1/self.assets for i in range(self.assets)] 
        self.wmamr20_portfolio = [1/self.assets for i in range(self.assets)] 
        self.rprt5_portfolio = [1/self.assets for i in range(self.assets)]  
        self.rprt10_portfolio = [1/self.assets for i in range(self.assets)]
        self.rprt15_portfolio = [1/self.assets for i in range(self.assets)] 
        self.rprt20_portfolio = [1/self.assets for i in range(self.assets)] 
        self.cwmr_portfolio = [1/self.assets for i in range(self.assets)]
        self.cash_agent_portfolio = [0 for i in range(self.assets)]  
        self.tco1_portfolio = [1/self.assets for i in range(self.assets)] 
        self.tco2_5_portfolio = [1/self.assets for i in range(self.assets)]
        self.tco2_10_portfolio = [1/self.assets for i in range(self.assets)]  
        self.tco2_15_portfolio = [1/self.assets for i in range(self.assets)] 
        self.tco2_20_portfolio = [1/self.assets for i in range(self.assets)]

    def geometric_mean(self, x):
        x = np.asarray(x) 
        return x.prod() ** (1/len(x)) 
    
    def weighted_geometric_mean(self, x, decay_factor=1.5):
        x = np.asarray(x) 
        w = [1/np.power(decay_factor,i) for i in range(len(x))] 
        prod = 1
        for i in range(len(x)): 
            prod *= np.power(x[i], w[i])
        return prod ** (1/np.sum(w))  
    
    def get_returns(self): 
        returns = [] 
        if self.rtypes == "1" or self.rtypes == "10": 
            for ticker in self.tickers:
                while True: 
                    try: 
                        df = pyupbit.get_ohlcv(ticker, interval='minute60') 
                        close = df['open'].values 
                        close = close[self.r-1::self.r]
                        ret = close[-1] / close[-2] 
                        returns.append(ret) 
                        break 
                    except Exception as e: 
                        print(e)
                        continue  
        elif self.rtypes == "0.5": 
            for ticker in self.tickers:
                while True: 
                    try: 
                        df = pyupbit.get_ohlcv(ticker, interval='minute30') 
                        close = df['open'].values 
                        ret = close[-1] / close[-2] 
                        returns.append(ret) 
                        break 
                    except Exception as e: 
                        print(e)
                        continue  
        elif self.rtypes == "4": 
            for ticker in self.tickers:
                while True: 
                    try: 
                        df = pyupbit.get_ohlcv(ticker, interval='minute240') 
                        close = df['open'].values 
                        ret = close[-1] / close[-2] 
                        returns.append(ret) 
                        break 
                    except Exception as e: 
                        print(e)
                        continue  
        return returns 
                
    def predict(self): 
        # store historical profits 
        if self.iterations > 0:
            # store historical profits for GMA model selector
            returns = self.get_returns() # get returns according to rtypes 
            self.pamr_history.append(np.dot(self.pamr_portfolio, returns)) 
            self.olmar5_history.append(np.dot(self.olmar5_portfolio, returns)) 
            self.olmar10_history.append(np.dot(self.olmar10_portfolio, returns)) 
            self.olmar15_history.append(np.dot(self.olmar15_portfolio, returns)) 
            self.olmar20_history.append(np.dot(self.olmar20_portfolio, returns)) 
            self.olvwma5_history.append(np.dot(self.olvwma5_portfolio, returns)) 
            self.olvwma10_history.append(np.dot(self.olvwma10_portfolio, returns)) 
            self.olvwma15_history.append(np.dot(self.olvwma15_portfolio, returns))
            self.olvwma20_history.append(np.dot(self.olvwma20_portfolio, returns)) 
            self.ons_history.append(np.dot(self.ons_portfolio, returns))
            self.eg_history.append(np.dot(self.eg_portfolio, returns)) 
            self.uniform_crp_history.append(np.dot(self.uniform_crp_portfolio, returns))
            self.wmamr5_history.append(np.dot(self.wmamr5_portfolio, returns)) 
            self.wmamr10_history.append(np.dot(self.wmamr10_portfolio, returns)) 
            self.wmamr15_history.append(np.dot(self.wmamr15_portfolio, returns)) 
            self.wmamr20_history.append(np.dot(self.wmamr20_portfolio, returns)) 
            self.rprt5_history.append(np.dot(self.rprt5_portfolio, returns)) 
            self.rprt10_history.append(np.dot(self.rprt10_portfolio, returns))  
            self.rprt15_history.append(np.dot(self.rprt15_portfolio, returns)) 
            self.rprt20_history.append(np.dot(self.rprt20_portfolio, returns)) 
            self.cwmr_history.append(np.dot(self.cwmr_portfolio, returns)) 
            self.tco1_history.append(np.dot(self.tco1_portfolio, returns)) 
            self.tco2_5_history.append(np.dot(self.tco2_5_portfolio, returns)) 
            self.tco2_10_history.append(np.dot(self.tco2_10_portfolio, returns))  
            self.tco2_15_history.append(np.dot(self.tco2_15_portfolio, returns)) 
            self.tco2_20_history.append(np.dot(self.tco2_20_portfolio, returns)) 
            self.rmr5_history.append(np.dot(self.rmr5_portfolio, returns))
            self.rmr10_history.append(np.dot(self.rmr10_portfolio, returns)) 
            self.rmr15_history.append(np.dot(self.rmr15_portfolio, returns)) 
            self.rmr20_history.append(np.dot(self.rmr20_portfolio, returns)) 
            # sliding window for history 
            if len(self.pamr_history) > self.gma_window_size: 
                self.pamr_history.pop(0)
            if len(self.olmar5_history) > self.gma_window_size: 
                self.olmar5_history.pop(0) 
            if len(self.olmar10_history) > self.gma_window_size: 
                self.olmar10_history.pop(0)
            if len(self.olmar15_history) > self.gma_window_size: 
                self.olmar15_history.pop(0)
            if len(self.olmar20_history) > self.gma_window_size: 
                self.olmar20_history.pop(0)
            if len(self.olvwma5_history) > self.gma_window_size: 
                self.olvwma5_history.pop(0)
            if len(self.olvwma10_history) > self.gma_window_size: 
                self.olvwma10_history.pop(0)
            if len(self.olvwma15_history) > self.gma_window_size: 
                self.olvwma15_history.pop(0)
            if len(self.olvwma20_history) > self.gma_window_size: 
                self.olvwma20_history.pop(0)
            if len(self.ons_history) > self.gma_window_size: 
                self.ons_history.pop(0)
            if len(self.eg_history) > self.gma_window_size: 
                self.eg_history.pop(0)
            if len(self.uniform_crp_history) > self.gma_window_size: 
                self.uniform_crp_history.pop(0) 
            if len(self.wmamr5_history) > self.gma_window_size: 
                self.wmamr5_history.pop(0) 
            if len(self.wmamr10_history) > self.gma_window_size: 
                self.wmamr10_history.pop(0) 
            if len(self.wmamr15_history) > self.gma_window_size: 
                self.wmamr15_history.pop(0) 
            if len(self.wmamr20_history) > self.gma_window_size: 
                self.wmamr20_history.pop(0) 
            if len(self.rprt5_history) > self.gma_window_size: 
                self.rprt5_history.pop(0) 
            if len(self.rprt10_history) > self.gma_window_size: 
                self.rprt10_history.pop(0) 
            if len(self.rprt15_history) > self.gma_window_size: 
                self.rprt15_history.pop(0)  
            if len(self.rprt20_history) > self.gma_window_size: 
                self.rprt20_history.pop(0)  
            if len(self.cwmr_history) > self.gma_window_size: 
                self.cwmr_history.pop(0)  
            if len(self.tco1_history) > self.gma_window_size: 
                self.tco1_history.pop(0) 
            if len(self.tco2_5_history) > self.gma_window_size: 
                self.tco2_5_history.pop(0) 
            if len(self.tco2_10_history) > self.gma_window_size: 
                self.tco2_10_history.pop(0)
            if len(self.tco2_15_history) > self.gma_window_size: 
                self.tco2_15_history.pop(0) 
            if len(self.tco2_20_history) > self.gma_window_size: 
                self.tco2_20_history.pop(0) 
            if len(self.rmr5_history) > self.gma_window_size:  
                self.rmr5_history.pop(0) 
            if len(self.rmr10_history) > self.gma_window_size: 
                self.rmr10_history.pop(0) 
            if len(self.rmr15_history) > self.gma_window_size:  
                self.rmr15_history.pop(0) 
            if len(self.rmr20_history) > self.gma_window_size: 
                self.rmr20_history.pop(0)

    
            # store historical profits for wgma model selector 
            self.wgma_pamr_history.append(np.dot(self.pamr_portfolio, returns)) 
            self.wgma_olmar5_history.append(np.dot(self.olmar5_portfolio, returns)) 
            self.wgma_olmar10_history.append(np.dot(self.olmar10_portfolio, returns)) 
            self.wgma_olmar15_history.append(np.dot(self.olmar15_portfolio, returns)) 
            self.wgma_olmar20_history.append(np.dot(self.olmar20_portfolio, returns)) 
            self.wgma_olvwma5_history.append(np.dot(self.olvwma5_portfolio, returns)) 
            self.wgma_olvwma10_history.append(np.dot(self.olvwma10_portfolio, returns)) 
            self.wgma_olvwma15_history.append(np.dot(self.olvwma15_portfolio, returns))
            self.wgma_olvwma20_history.append(np.dot(self.olvwma20_portfolio, returns)) 
            self.wgma_ons_history.append(np.dot(self.ons_portfolio, returns))
            self.wgma_eg_history.append(np.dot(self.eg_portfolio, returns)) 
            self.wgma_uniform_crp_history.append(np.dot(self.uniform_crp_portfolio, returns))  
            self.wgma_wmamr5_history.append(np.dot(self.wmamr5_portfolio, returns)) 
            self.wgma_wmamr10_history.append(np.dot(self.wmamr10_portfolio, returns)) 
            self.wgma_wmamr15_history.append(np.dot(self.wmamr15_portfolio, returns)) 
            self.wgma_wmamr20_history.append(np.dot(self.wmamr20_portfolio, returns)) 
            self.wgma_rprt5_history.append(np.dot(self.rprt5_portfolio, returns)) 
            self.wgma_rprt10_history.append(np.dot(self.rprt10_portfolio, returns)) 
            self.wgma_rprt15_history.append(np.dot(self.rprt15_portfolio, returns))
            self.wgma_rprt20_history.append(np.dot(self.rprt20_portfolio, returns)) 
            self.wgma_cwmr_history.append(np.dot(self.cwmr_portfolio, returns)) 
            self.wgma_tco1_history.append(np.dot(self.tco1_portfolio, returns)) 
            self.wgma_tco2_5_history.append(np.dot(self.tco2_5_portfolio, returns)) 
            self.wgma_tco2_10_history.append(np.dot(self.tco2_10_portfolio, returns))
            self.wgma_tco2_15_history.append(np.dot(self.tco2_15_portfolio, returns)) 
            self.wgma_tco2_20_history.append(np.dot(self.tco2_20_portfolio, returns))  
            self.wgma_rmr5_history.append(np.dot(self.rmr5_portfolio, returns))
            self.wgma_rmr10_history.append(np.dot(self.rmr10_portfolio, returns)) 
            self.wgma_rmr15_history.append(np.dot(self.rmr15_portfolio, returns)) 
            self.wgma_rmr20_history.append(np.dot(self.rmr20_portfolio, returns)) 
            # sliding window for history 
            if len(self.wgma_pamr_history) > self.wgma_window_size: 
                self.wgma_pamr_history.pop(0)
            if len(self.wgma_olmar5_history) > self.wgma_window_size: 
                self.wgma_olmar5_history.pop(0) 
            if len(self.wgma_olmar10_history) > self.wgma_window_size: 
                self.wgma_olmar10_history.pop(0)
            if len(self.wgma_olmar15_history) > self.wgma_window_size: 
                self.wgma_olmar15_history.pop(0)
            if len(self.wgma_olmar20_history) > self.wgma_window_size: 
                self.wgma_olmar20_history.pop(0)
            if len(self.wgma_olvwma5_history) > self.wgma_window_size: 
                self.wgma_olvwma5_history.pop(0)
            if len(self.wgma_olvwma10_history) > self.wgma_window_size: 
                self.wgma_olvwma10_history.pop(0)
            if len(self.wgma_olvwma15_history) > self.wgma_window_size: 
                self.wgma_olvwma15_history.pop(0)
            if len(self.wgma_olvwma20_history) > self.wgma_window_size: 
                self.wgma_olvwma20_history.pop(0)
            if len(self.wgma_ons_history) > self.wgma_window_size: 
                self.wgma_ons_history.pop(0)
            if len(self.wgma_eg_history) > self.wgma_window_size: 
                self.wgma_eg_history.pop(0)
            if len(self.wgma_uniform_crp_history) > self.wgma_window_size: 
                self.wgma_uniform_crp_history.pop(0) 
            if len(self.wgma_wmamr5_history) > self.wgma_window_size: 
                self.wgma_wmamr5_history.pop(0) 
            if len(self.wgma_wmamr10_history) > self.wgma_window_size: 
                self.wgma_wmamr10_history.pop(0)  
            if len(self.wgma_wmamr15_history) > self.wgma_window_size:  
                self.wgma_wmamr15_history.pop(0) 
            if len(self.wgma_wmamr20_history) > self.wgma_window_size:  
                self.wgma_wmamr20_history.pop(0)
            if len(self.wgma_rprt5_history) > self.wgma_window_size: 
                self.wgma_rprt5_history.pop(0) 
            if len(self.wgma_rprt10_history) > self.wgma_window_size: 
                self.wgma_rprt10_history.pop(0)   
            if len(self.wgma_rprt15_history) > self.wgma_window_size: 
                self.wgma_rprt15_history.pop(0) 
            if len(self.wgma_rprt20_history) > self.wgma_window_size: 
                self.wgma_rprt20_history.pop(0) 
            if len(self.wgma_cwmr_history) > self.wgma_window_size:  
                self.wgma_cwmr_history.pop(0) 
            if len(self.wgma_tco1_history) > self.wgma_window_size: 
                self.wgma_tco1_history.pop(0) 
            if len(self.wgma_tco2_5_history) > self.wgma_window_size: 
                self.wgma_tco2_5_history.pop(0) 
            if len(self.wgma_tco2_10_history) > self.wgma_window_size: 
                self.wgma_tco2_10_history.pop(0)  
            if len(self.wgma_tco2_15_history) > self.wgma_window_size: 
                self.wgma_tco2_15_history.pop(0) 
            if len(self.wgma_tco2_20_history) > self.wgma_window_size: 
                self.wgma_tco2_20_history.pop(0) 
            if len(self.wgma_rmr5_history) > self.wgma_window_size:  
                self.wgma_rmr5_history.pop(0) 
            if len(self.wgma_rmr10_history) > self.wgma_window_size: 
                self.wgma_rmr10_history.pop(0) 
            if len(self.wgma_rmr15_history) > self.wgma_window_size:  
                self.wgma_rmr15_history.pop(0) 
            if len(self.wgma_rmr20_history) > self.wgma_window_size:  
                self.wgma_rmr20_history.pop(0) 
        
        # make predictions for portfolio weights
        self.pamr_portfolio = self.pamr.predict(self.pamr_portfolio,self.r,self.rtypes) 
        self.olmar5_portfolio = self.olmar5.predict(self.olmar5_portfolio,self.r,self.rtypes)
        self.olmar10_portfolio = self.olmar10.predict(self.olmar10_portfolio,self.r,self.rtypes) 
        self.olmar15_portfolio = self.olmar15.predict(self.olmar15_portfolio,self.r,self.rtypes) 
        self.olmar20_portfolio = self.olmar20.predict(self.olmar20_portfolio,self.r,self.rtypes)
        self.olvwma5_portfolio = self.olvwma5.predict(self.olvwma5_portfolio,self.r,self.rtypes) 
        self.olvwma10_portfolio = self.olvwma10.predict(self.olvwma10_portfolio,self.r,self.rtypes)
        self.olvwma15_portfolio = self.olvwma15.predict(self.olvwma15_portfolio,self.r,self.rtypes) 
        self.olvwma20_portfolio = self.olvwma20.predict(self.olvwma20_portfolio,self.r,self.rtypes) 
        self.ons_portfolio = self.ons.predict(self.ons_portfolio,self.r,self.rtypes)
        self.eg_portfolio = self.eg.predict(self.eg_portfolio,self.r,self.rtypes)
        self.uniform_crp_portfolio = self.uniform_crp.predict(self.uniform_crp_portfolio)
        self.cash_agent_portfolio = self.cash_agent.predict(self.cash_agent_portfolio) 
        self.wmamr5_portfolio = self.wmamr5.predict(self.wmamr5_portfolio,self.r,self.rtypes) 
        self.wmamr10_portfolio = self.wmamr10.predict(self.wmamr10_portfolio,self.r,self.rtypes) 
        self.wmamr15_portfolio = self.wmamr15.predict(self.wmamr15_portfolio,self.r,self.rtypes) 
        self.wmamr20_portfolio = self.wmamr20.predict(self.wmamr20_portfolio,self.r,self.rtypes) 
        self.rprt5_portfolio = self.rprt5.predict(self.rprt5_portfolio,self.r,self.rtypes) 
        self.rprt10_portfolio = self.rprt10.predict(self.rprt10_portfolio,self.r,self.rtypes) 
        self.rprt15_portfolio = self.rprt15.predict(self.rprt15_portfolio,self.r,self.rtypes) 
        self.rprt20_portfolio = self.rprt20.predict(self.rprt20_portfolio,self.r,self.rtypes) 
        self.cwmr_portfolio = self.cwmr.predict(self.cwmr_portfolio,self.r,self.rtypes)  
        self.tco1_portfolio = self.tco1.predict(self.tco1_portfolio,self.r,self.rtypes) 
        self.tco2_5_portfolio = self.tco2_5.predict(self.tco2_5_portfolio,self.r,self.rtypes) 
        self.tco2_10_portfolio = self.tco2_10.predict(self.tco2_10_portfolio,self.r,self.rtypes)  
        self.tco2_15_portfolio = self.tco2_15.predict(self.tco2_15_portfolio,self.r,self.rtypes)
        self.tco2_20_portfoiio = self.tco2_20.predict(self.tco2_20_portfolio,self.r,self.rtypes) 
        self.rmr5_portfolio = self.rmr5.predict(self.rmr5_portfolio, self.r, self.rtypes) 
        self.rmr10_portfolio = self.rmr10.predict(self.rmr10_portfolio, self.r, self.rtypes) 
        self.rmr15_portfolio = self.rmr15.predict(self.rmr15_portfolio, self.r, self.rtypes) 
        self.rmr20_portfolio = self.rmr20.predict(self.rmr20_portfolio, self.r, self.rtypes) 
        
        
        model_portfolios = [self.pamr_portfolio, 
                            self.olmar5_portfolio,
                            self.olmar10_portfolio,
                            self.olmar15_portfolio,
                            self.olmar20_portfolio,
                            self.olvwma5_portfolio,
                            self.olvwma10_portfolio,
                            self.olvwma15_portfolio,
                            self.olvwma20_portfolio,
                            self.ons_portfolio,
                            self.eg_portfolio,
                            self.uniform_crp_portfolio, 
                            self.wmamr5_portfolio, 
                            self.wmamr10_portfolio,  
                            self.wmamr15_portfolio, 
                            self.wmamr20_portfolio,  
                            self.rprt5_portfolio, 
                            self.rprt10_portfolio,  
                            self.rprt15_portfolio,  
                            self.rprt20_portfolio, 
                            self.cwmr_portfolio, 
                            self.tco1_portfolio, 
                            self.tco2_5_portfolio, 
                            self.tco2_10_portfolio, 
                            self.tco2_15_portfolio,
                            self.tco2_20_portfolio,
                            self.rmr5_portfolio, 
                            self.rmr10_portfolio, 
                            self.rmr15_portfolio, 
                            self.rmr20_portfolio]  
                
        model_names = ["PAMR", 
                       "OLMAR5", 
                       "OLMAR10", 
                       "OLMAR15", 
                       "OLMAR20", 
                       "OLVWMA5", 
                       "OLVWMA10", 
                       "OLVWMA15", 
                       "OLVWMA20", 
                       "ONS", 
                       "EG", 
                       "UNIFORM CRP", 
                       "WMAMR5", 
                       "WMAMR10", 
                       "WMAMR15", 
                       "WMAMR20", 
                       "RPRT5", 
                       "RPRT10", 
                       "RPRT15", 
                       "RPRT20", 
                       "CWMR",
                       "TCO1", 
                       "TCO2_5", 
                       "TCO2_10", 
                       "TCO2_15", 
                       "TCO2_20", 
                       "RMR5", 
                       "RMR10", 
                       "RMR15", 
                       "RMR20"] 
                
        
        if self.iterations == 0: 
            # choose ons for first trade  
            print("First trade {} choosing ONS".format(self.name))
            self.iterations += 1 
            return self.ons_portfolio 
        else: 
            self.iterations += 1 
            # choose base model with highest cumulative returns 
            wgma_pamr = self.weighted_geometric_mean(self.wgma_pamr_history) 
            wgma_olmar5 = self.weighted_geometric_mean(self.wgma_olmar5_history) 
            wgma_olmar10 = self.weighted_geometric_mean(self.wgma_olmar10_history) 
            wgma_olmar15 = self.weighted_geometric_mean(self.wgma_olmar15_history) 
            wgma_olmar20 = self.weighted_geometric_mean(self.wgma_olmar20_history) 
            wgma_olvwma5 = self.weighted_geometric_mean(self.wgma_olvwma5_history) 
            wgma_olvwma10 = self.weighted_geometric_mean(self.wgma_olvwma10_history) 
            wgma_olvwma15 = self.weighted_geometric_mean(self.wgma_olvwma15_history) 
            wgma_olvwma20 = self.weighted_geometric_mean(self.wgma_olvwma20_history)
            wgma_ons = self.weighted_geometric_mean(self.wgma_ons_history) 
            wgma_eg = self.weighted_geometric_mean(self.wgma_eg_history) 
            wgma_uniform_crp = self.weighted_geometric_mean(self.wgma_uniform_crp_history)  
            wgma_wmamr5 = self.weighted_geometric_mean(self.wgma_wmamr5_history) 
            wgma_wmamr10 = self.weighted_geometric_mean(self.wgma_wmamr10_history) 
            wgma_wmamr15 = self.weighted_geometric_mean(self.wgma_wmamr15_history) 
            wgma_wmamr20 = self.weighted_geometric_mean(self.wgma_wmamr20_history) 
            wgma_rprt5 = self.weighted_geometric_mean(self.wgma_rprt5_history) 
            wgma_rprt10 = self.weighted_geometric_mean(self.wgma_rprt10_history)  
            wgma_rprt15 = self.weighted_geometric_mean(self.wgma_rprt15_history) 
            wgma_rprt20 = self.weighted_geometric_mean(self.wgma_rprt20_history) 
            wgma_cwmr = self.weighted_geometric_mean(self.wgma_cwmr_history)  
            wgma_tco1 = self.weighted_geometric_mean(self.wgma_tco1_history) 
            wgma_tco2_5 = self.weighted_geometric_mean(self.wgma_tco2_5_history) 
            wgma_tco2_10 = self.weighted_geometric_mean(self.wgma_tco2_10_history)  
            wgma_tco2_15 = self.weighted_geometric_mean(self.wgma_tco2_15_history) 
            wgma_tco2_20 = self.weighted_geometric_mean(self.wgma_tco2_20_history) 
            wgma_rmr5 = self.weighted_geometric_mean(self.wgma_rmr5_history) 
            wgma_rmr10 = self.weighted_geometric_mean(self.wgma_rmr10_history) 
            wgma_rmr15 = self.weighted_geometric_mean(self.wgma_rmr15_history) 
            wgma_rmr20 = self.weighted_geometric_mean(self.wgma_rmr20_history) 
            
            wgma_profits = [wgma_pamr,
                            wgma_olmar5,
                            wgma_olmar10, 
                            wgma_olmar15, 
                            wgma_olmar20, 
                            wgma_olvwma5, 
                            wgma_olvwma10, 
                            wgma_olvwma15, 
                            wgma_olvwma20,
                            wgma_ons, 
                            wgma_eg, 
                            wgma_uniform_crp, 
                            wgma_wmamr5,
                            wgma_wmamr10, 
                            wgma_wmamr15,
                            wgma_wmamr20, 
                            wgma_rprt5,
                            wgma_rprt10, 
                            wgma_rprt15,
                            wgma_rprt20, 
                            wgma_cwmr, 
                            wgma_tco1, 
                            wgma_tco2_5, 
                            wgma_tco2_10,
                            wgma_tco2_15,
                            wgma_tco2_20,
                            wgma_rmr5,
                            wgma_rmr10, 
                            wgma_rmr15, 
                            wgma_rmr20]  
            
            best_wgma_profits, best_wgma_idx = np.max(wgma_profits), np.argmax(wgma_profits)
            
            if best_wgma_profits <= 1.0: 
                print("{} choosing cash agent".format(self.name))
                return self.cash_agent_portfolio   
            else: 
                # select base model with the best gma profits
                gma_pamr = self.geometric_mean(self.pamr_history)
                gma_olmar5 = self.geometric_mean(self.olmar5_history) 
                gma_olmar10 = self.geometric_mean(self.olmar10_history) 
                gma_olmar15 = self.geometric_mean(self.olmar15_history) 
                gma_olmar20 = self.geometric_mean(self.olmar20_history)
                gma_olvwma5 = self.geometric_mean(self.olvwma5_history) 
                gma_olvwma10 = self.geometric_mean(self.olvwma10_history) 
                gma_olvwma15 = self.geometric_mean(self.olvwma15_history) 
                gma_olvwma20 = self.geometric_mean(self.olvwma20_history) 
                gma_ons = self.geometric_mean(self.ons_history)
                gma_eg = self.geometric_mean(self.eg_history) 
                gma_uniform_crp = self.geometric_mean(self.uniform_crp_history) 
                gma_wmamr5 = self.geometric_mean(self.wmamr5_history) 
                gma_wmamr10 = self.geometric_mean(self.wmamr10_history)  
                gma_wmamr15 = self.geometric_mean(self.wmamr15_history) 
                gma_wmamr20 = self.geometric_mean(self.wmamr20_history) 
                gma_rprt5 = self.geometric_mean(self.rprt5_history) 
                gma_rprt10 = self.geometric_mean(self.rprt10_history)  
                gma_rprt15 = self.geometric_mean(self.rprt15_history) 
                gma_rprt20 = self.geometric_mean(self.rprt20_history)  
                gma_cwmr = self.geometric_mean(self.cwmr_history) 
                gma_tco1 = self.geometric_mean(self.tco1_history) 
                gma_tco2_5 = self.geometric_mean(self.tco2_5_history) 
                gma_tco2_10 = self.geometric_mean(self.tco2_10_history) 
                gma_tco2_15 = self.geometric_mean(self.tco2_15_history) 
                gma_tco2_20 = self.geometric_mean(self.tco2_20_history) 
                gma_rmr5 = self.geometric_mean(self.rmr5_history) 
                gma_rmr10 = self.geometric_mean(self.rmr10_history) 
                gma_rmr15 = self.geometric_mean(self.rmr15_history) 
                gma_rmr20 = self.geometric_mean(self.rmr20_history) 
                
                gma_profits = [gma_pamr, 
                               gma_olmar5, 
                               gma_olmar10, 
                               gma_olmar15, 
                               gma_olmar20, 
                               gma_olvwma5,
                               gma_olvwma10,
                               gma_olvwma15,
                               gma_olvwma20,
                               gma_ons,
                               gma_eg,
                               gma_uniform_crp, 
                               gma_wmamr5, 
                               gma_wmamr10, 
                               gma_wmamr15, 
                               gma_wmamr20, 
                               gma_rprt5, 
                               gma_rprt10,
                               gma_rprt15,
                               gma_rprt20, 
                               gma_cwmr, 
                               gma_tco1, 
                               gma_tco2_5, 
                               gma_tco2_10, 
                               gma_tco2_15, 
                               gma_tco2_20, 
                               gma_rmr5, 
                               gma_rmr10, 
                               gma_rmr15,  
                               gma_rmr20] 
                
               
                best_gma_profits, best_gma_idx = np.max(gma_profits), np.argmax(gma_profits)  
                
                if best_gma_profits <= 1.0: 
                    print("{} choosing cash agent".format(self.name))
                    return self.cash_agent_portfolio 
                else: 
                    print("{} choosing {}".format(self.name, model_names[best_gma_idx]))
                    return model_portfolios[best_gma_idx]
                
            

In [277]:
tickers = ["KRW-BTC", 
           "KRW-ADA", 
           "KRW-XRP", 
           "KRW-DOT", 
           "KRW-DOGE", 
           "KRW-AXS", 
           "KRW-BCH", 
           "KRW-SRM", 
           "KRW-TRX", 
           "KRW-ETH", 
           "KRW-XEC", 
           "KRW-POLY", 
           "KRW-OMG", 
           "KRW-ONG", 
           "KRW-XTZ", 
           "KRW-HIVE", 
           "KRW-LINK", 
           "KRW-XLM", 
           "KRW-EOS", 
           "KRW-ETC"] 
# API keys 
access_key = "ZcG39GRdKaD1jPWS9uk4xZxxWtSAlA92yox1xm5A"
secret_key = "Y4I2SCa0HwGfLKxG2MtuZfiyCKlHaW2M3ljOm7Vs"
upbit = pyupbit.Upbit(access_key, secret_key)

short_term_agent = Agent(r=1,rtypes="1",tickers=tickers, wgma_window_size=10, gma_window_size=20, name="agent_60", gamma=0.05)
short_term_agent.predict() 


100%|██████████| 20/20 [00:02<00:00,  6.91it/s]

First trade agent_60 choosing ONS


array([0.05436444, 0.05386985, 0.05201951, 0.0432994 , 0.05229922,
       0.02672785, 0.05241302, 0.02180925, 0.05803311, 0.05289782,
       0.05310196, 0.04761519, 0.0585097 , 0.05803311, 0.05009471,
       0.05310853, 0.05355311, 0.04969878, 0.05364336, 0.05490807])

In [278]:
agent30 = Agent(r=0.5,rtypes="0.5",tickers=tickers, wgma_window_size=10, gma_window_size=20, name="agent_30", gamma=0.05)
agent30.predict() 


100%|██████████| 20/20 [00:08<00:00,  2.48it/s]

First trade agent_30 choosing ONS


array([0.04934348, 0.04517181, 0.04936964, 0.04897375, 0.04936964,
       0.06654057, 0.04775373, 0.05313477, 0.04936964, 0.04764592,
       0.04936964, 0.05023771, 0.05904099, 0.04680111, 0.05030657,
       0.04837756, 0.04390377, 0.04936964, 0.04715722, 0.04876284])

In [279]:
agent240 = Agent(r=4,rtypes="4",tickers=tickers, wgma_window_size=10, gma_window_size=20, name="agent_240", gamma=0.05)
agent240.predict() 


100%|██████████| 20/20 [00:08<00:00,  2.49it/s]


First trade agent_240 choosing ONS


array([5.52510301e-02, 5.99832068e-02, 5.47821035e-02, 4.47483708e-02,
       5.74493821e-02, 3.23362823e-07, 5.20478431e-02, 3.32268663e-02,
       5.82887068e-02, 5.23853952e-02, 5.18209752e-02, 4.31872679e-02,
       5.03920601e-02, 4.15804456e-02, 6.80357554e-02, 5.85853921e-02,
       5.04165558e-02, 4.56076110e-02, 5.54282210e-02, 6.67824879e-02])

In [280]:
agent600 = Agent(r=10,rtypes="10",tickers=tickers, wgma_window_size=10, gma_window_size=20, name="agent_600", gamma=0.05) 
agent600.predict() 

100%|██████████| 20/20 [00:02<00:00,  7.37it/s]

First trade agent_600 choosing ONS


array([3.26202940e-02, 5.54042069e-02, 4.25624019e-02, 3.19657562e-02,
       5.02683443e-02, 1.14846794e-09, 4.71116637e-02, 2.65269671e-02,
       5.54410240e-02, 3.92577988e-02, 7.38813040e-02, 1.00268673e-01,
       6.94902558e-02, 4.33698439e-02, 9.01948140e-02, 9.42752390e-02,
       2.84917965e-02, 4.21728987e-02, 4.65622132e-02, 3.01345035e-02])

In [282]:
class Agent_Selector:
    def __init__(self, tickers, gma_window_size, wgma_window_size, upbit, gamma, threshold):
        self.tickers = tickers
        self.assets = len(self.tickers)
        self.iterations = 0 
        self.upbit = upbit # upbit account for actual trading 
        self.gamma = gamma  
        self.threshold = threshold
        self.previous = 600 # 600: 10hrs, 240: 4hrs, 60: 1hrs, 30: 0.5hrs
        # agents 
        self.agent600 = Agent(r=10, rtypes="10", tickers=self.tickers, wgma_window_size=10, gma_window_size=20, name="agent_600", gamma=self.gamma) 
        self.agent240 = Agent(r=4, rtypes="4", tickers=self.tickers, wgma_window_size=10, gma_window_size=20, name="agent_240", gamma=self.gamma) 
        self.agent60 = Agent(r=1, rtypes="1", tickers=self.tickers, wgma_window_size=10, gma_window_size=20, name="agent_60", gamma=self.gamma) 
        self.agent30 = Agent(r=0.5, rtypes="0.5", tickers=self.tickers, wgma_window_size=10, gma_window_size=20, name="agent_30", gamma=self.gamma) 
        self.gma_window_size = gma_window_size   
        self.wgma_window_size = wgma_window_size 
        # portfolios 
        self.agent600_portfolio = [1/self.assets for i in range(self.assets)] 
        self.agent240_portfolio = [1/self.assets for i in range(self.assets)] 
        self.agent60_portfolio = [1/self.assets for i in range(self.assets)]
        self.agent30_portfolio = [1/self.assets for i in range(self.assets)] 
        self.cash_agent_portfolio = [0 for i in range(self.assets)]
        # other information 
        self.agent600_history = [] 
        self.wgma_agent600_history = [] 
        self.agent240_history = [] 
        self.wgma_agent240_history = [] 
        self.agent60_history = [] 
        self.wgma_agent60_history = [] 
        self.agent30_history = [] 
        self.wgma_agent30_history = []
        self.cash_status = [] 
        self.agent_names = ["agent600", "agent240", "agent60", "agent30"] 
        
    def geometric_mean(self, x):
        x = np.asarray(x) 
        return x.prod() ** (1/len(x))
    
    def weighted_geometric_mean(self, x, decay_factor=1.5):
        x = np.asarray(x) 
        w = [1/np.power(decay_factor,i) for i in range(len(x))] 
        prod = 1
        for i in range(len(x)): 
            prod *= np.power(x[i], w[i])
        return prod ** (1/np.sum(w))  

    
    ''' 
    The returns have to obtained every 30 minutes, and the returns for every 30 minutes is used for comparison
    '''
    def get_returns(self): 
        returns = []
        for ticker in self.tickers:
            while True: 
                try: 
                    df = pyupbit.get_ohlcv(ticker, interval='minute30')
                    close = df['open'].values 
                    ret = close[-1] / close[-2] 
                    returns.append(ret) 
                    break 
                except Exception as e: 
                    print(e) 
                    continue 
        return returns
    
    ''' 
    Check if the current portfolio is cash agent 
    '''
    def check_zero_portfolio(self, arr): 
        for i in range(len(arr)): 
            if arr[i] != 0: 
                return False 
        return True 
    
    '''
    The trade function will be called every 30 minutes 
    In general, it will be called according to the shortest frequency rebalancing time 
    '''
    def trade(self):        
        if self.iterations == 0:
            # get cash amount 
            cash_amount = self.upbit.get_balance("KRW")
            print("current cash amount = {:.2f} won".format(cash_amount)) 
            self.cash_status.append(cash_amount)
            # open according to the long portfolio 
            self.agent600_portfolio = self.agent600.predict() 
            self.agent240_portfolio = self.agent240.predict()  
            self.agent60_portfolio = self.agent60.predict() 
            self.agent30_portfolio = self.agent30.predict() 
            self.iterations += 1
            print("initially opening positions according to agent600 ...")
            for i in range(self.assets):
                while True: 
                    try: 
                        if self.agent600_portfolio[i] > 0: 
                            self.upbit.buy_market_order(self.tickers[i], cash_amount*(1-self.gamma/100)*self.agent600_portfolio[i])
                            time.sleep(1)
                        break  
                    except Exception as e: 
                        print(e) 
                        continue 
                        
        elif self.iterations > 0:
            # store historical profits 
            returns = self.get_returns()
            ''' Store agent 600 profits '''
            if self.check_zero_portfolio(self.agent600_portfolio) == True:
                print("agent 600 previously predicted cash agent!")
                self.agent600_history.append(1.0)
                self.wgma_agent600_history.append(1.0) 
            else: 
                agent600_profits = np.dot(self.agent600_portfolio, returns)  
                self.agent600_history.append(agent600_profits)   
                self.wgma_agent600_history.append(agent600_profits)  
            
            ''' Store agent 240 profits ''' 
            if self.check_zero_portfolio(self.agent240_portfolio) == True: 
                print("agent 240 previously predicted cash agent!")
                self.agent240_history.append(1.0) 
                self.wgma_agent240_history.append(1.0) 
            else: 
                agent240_profits = np.dot(self.agent240_portfolio, returns)
                # add penalty 
                agent240_profits = agent240_profits * (1-self.gamma/100) * (1-self.gamma/100)
                self.agent240_history.append(agent240_profits) 
                self.wgma_agent240_history.append(agent240_profits) 
                
            ''' Store agent 60 profits '''
            if self.check_zero_portfolio(self.agent60_portfolio) == True: 
                print("agent 60 previously predicted cash agent!") 
                self.agent60_history.append(1.0) 
                self.wgma_agent60_history.append(1.0) 
            else: 
                agent60_profits = np.dot(self.agent60_portfolio, returns)  
                # add penalty 
                agent60_profits = agent60_profits * (1-self.gamma/100) * (1-self.gamma/100) * (1-self.gamma/100) * (1-self.gamma/100)
                self.agent60_history.append(agent60_profits) 
                self.wgma_agent60_history.append(agent60_profits) 
                
            ''' Store agent 30 profits ''' 
            if self.check_zero_portfolio(self.agent30_portfolio) == True:  
                print("agent 30 previously predicted cash agent!") 
                self.agent30_history.append(1.0)
                self.wgma_agent30_history.append(1.0) 
            else: 
                agent30_profits = np.dot(self.agent30_portfolio, returns)  
                # add penalty 
                agent30_profits = agent30_profits * (1-self.gamma/100) * (1-self.gamma/100) * (1-self.gamma/100) * (1-self.gamma/100) * (1-self.gamma/100) * (1-self.gamma/100)
                self.agent30_history.append(agent30_profits) 
                self.wgma_agent30_history.append(agent30_profits)

    
            # sliding window effect for gma history 
            if len(self.agent600_history) > self.gma_window_size: 
                self.agent600_history.pop(0)
            if len(self.agent240_history) > self.gma_window_size: 
                self.agent240_history.pop(0) 
            if len(self.agent60_history) > self.gma_window_size: 
                self.agent60_history.pop(0) 
            if len(self.agent30_history) > self.gma_window_size: 
                self.agent30_history.pop(0) 
            
            # sliding window effect for wgma history 
            if len(self.wgma_agent600_history) > self.wgma_window_size: 
                self.wgma_agent600_history.pop(0)
            if len(self.wgma_agent240_history) > self.wgma_window_size: 
                self.wgma_agent240_history.pop(0) 
            if len(self.wgma_agent60_history) > self.wgma_window_size: 
                self.wgma_agent60_history.pop(0) 
            if len(self.wgma_agent30_history) > self.wgma_window_size: 
                self.wgma_agent30_history.pop(0) 

                
            ''' 
            If current period is the rebalancing period for any of the agents, then we must close position 
            and rebalance. 
            We have 
            10 hours (every 20th iteration) 
            4 hours (every 8th iteration) 
            1 hour (every 2nd iteration) 
            0.5 hours (every iteration)
            ''' 
            if self.previous == 600 and self.iterations%20 == 0: 
                print("agent600 was chosen so far and we are closing positions for it now...")
                for i in range(self.assets): 
                    while True: 
                        try: 
                            if self.agent600_portfolio[i] > 0: 
                                unit = self.upbit.get_balance(self.tickers[i]) 
                                self.upbit.sell_market_order(self.tickers[i], unit) 
                                time.sleep(1)
                            break 
                        except Exception as e: 
                            print(e)
                            continue 
                print("storing cash status")
                cash_amount = self.upbit.get_balance("KRW") 
                print("current cash amount = {:.2f} won".format(cash_amount)) 
                self.cash_status.append(cash_amount)
            elif self.previous == 240 and self.iterations%8 == 0: 
                print("agent240 was chosen so far and we are closing positions for it now...")
                for i in range(self.assets): 
                    while True: 
                        try: 
                            if self.agent240_portfolio[i] > 0: 
                                unit = self.upbit.get_balance(self.tickers[i])
                                self.upbit.sell_market_order(self.tickers[i], unit)  
                                time.sleep(1) 
                            break 
                        except Exception as e:  
                            print(e) 
                            continue 
                print("storing cash status")
                cash_amount = self.upbit.get_balance("KRW") 
                print("current cash amount = {:.2f} won".format(cash_amount)) 
                self.cash_status.append(cash_amount)
            elif self.previous == 60 and self.iterations%2 == 0: 
                print("agent60 was chosen so far and we are closing positions for it now...") 
                for i in range(self.assets): 
                    while True: 
                        try: 
                            if self.agent60_portfolio[i] > 0: 
                                unit = self.upbit.get_balance(self.tickers[i]) 
                                self.upbit.sell_market_order(self.tickers[i], unit) 
                                time.sleep(1) 
                            break 
                        except Exception as e: 
                            print(e)
                            continue 
                print("storing cash status")
                cash_amount = self.upbit.get_balance("KRW") 
                print("current cash amount = {:.2f} won".format(cash_amount)) 
                self.cash_status.append(cash_amount)
            elif self.previous == 30 and self.iterations%1 == 0: 
                print("agent30 was chosen so far and we are closing positions for it now...")
                for i in range(self.assets): 
                    while True: 
                        try: 
                            if self.agent30_portfolio[i] > 0: 
                                unit = self.upbit.get_balance(self.tickers[i]) 
                                self.upbit.sell_market_order(self.tickers[i], unit) 
                                time.sleep(1)
                            break 
                        except Exception as e: 
                            print(e) 
                            continue 
                print("storing cash status")
                cash_amount = self.upbit.get_balance("KRW") 
                print("current cash amount = {:.2f} won".format(cash_amount)) 
                self.cash_status.append(cash_amount)
                
                
            '''
            update predictions for all the agents 
            '''    
            if self.iterations%20 == 0: 
                self.agent600_portfolio = self.agent600.predict() 
            if self.iterations%8 == 0: 
                self.agent240_portfolio = self.agent240.predict() 
            if self.iterations%2 == 0:  
                self.agent60_portfolio = self.agent60.predict() 
            if self.iterations%1 == 0: 
                self.agent30_portfolio = self.agent30.predict() 
            
            '''
            Decide the best agent 
            '''
            wgma_agent600 = self.weighted_geometric_mean(self.wgma_agent600_history) 
            wgma_agent240 = self.weighted_geometric_mean(self.wgma_agent240_history) 
            wgma_agent60 = self.weighted_geometric_mean(self.wgma_agent60_history) 
            wgma_agent30 = self.weighted_geometric_mean(self.wgma_agent30_history) 
            
            wgma_profits = [wgma_agent600, wgma_agent240, wgma_agent60, wgma_agent30] 
            wgma_best_profits, wgma_best_idx = np.max(wgma_profits), np.argmax(wgma_profits) 
            
            print("agent600 wgma: {}".format(wgma_profits[0])) 
            print("agent240 wgma: {}".format(wgma_profits[1])) 
            print("agent60 wgma: {}".format(wgma_profits[2])) 
            print("agent30 wgma: {}".format(wgma_profits[3])) 
            

            ''' 
            If all the agents are doing badly, then we resort to cash agent for 30 minutes
            Also we have to close positions when cash is chosen 
            '''
            if wgma_best_profits <= 1.0: 
                print("choosing cash agent as all agents are doing badly") 
                for i in range(self.assets): 
                    while True: 
                        try: 
                            unit = self.upbit.get_balance(self.tickers[i]) 
                            self.upbit.sell_market_order(self.tickers[i], unit) 
                            time.sleep(1)
                            break 
                        except Exception as e: 
                            print(e) 
                            continue 
                self.previous = 30  
                return self.cash_agent_portfolio  
            elif wgma_best_profits > 1.0: 
                gma_agent600 = self.geometric_mean(self.agent600_history) 
                gma_agent240 = self.geometric_mean(self.agent240_history) 
                gma_agent60 = self.geometric_mean(self.agent60_history) 
                gma_agent30 = self.geometric_mean(self.agent30_history) 
                
                # gma_agent240 = gma_agent240 * (1-self.gamma/100) 
                # gma_agent60 = gma_agent60 * (1-self.gamma/100) * (1-self.gamma/100)
                # gma_agent30 = gma_agent30 * (1-self.gamma/100) * (1-self.gamma/100) * (1-self.gamma/100)  
            
                
                gma_profits = [gma_agent600, gma_agent240, gma_agent60, gma_agent30] 
                best_gma_profits, best_gma_idx = np.max(gma_profits), np.argmax(gma_profits) 
                
                
                print("agent600 gma: {}".format(gma_agent600)) 
                print("agent240 gma: {}".format(gma_agent240)) 
                print("agent60 gma: {}".format(gma_agent60))
                print("agent30 gma: {}".format(gma_agent30))
                
                
                
                if best_gma_profits > 1.0 + self.threshold:
                    print("choosing {}".format(self.agent_names[best_gma_idx]))  
                    
                    if best_gma_idx == 0 and (self.previous != 600 or self.iterations%20==0): 
                        for i in range(self.assets): 
                            while True: 
                                try: 
                                    unit = self.upbit.get_balance(self.tickers[i]) 
                                    self.upbit.sell_market_order(self.tickers[i], unit) 
                                    time.sleep(1)
                                    break 
                                except Exception as e: 
                                    print(e) 
                                    continue 
                        print("storing cash status")
                        cash_amount = self.upbit.get_balance("KRW") 
                        print("current cash amount = {:.2f} won".format(cash_amount)) 
                        self.cash_status.append(cash_amount)
                        self.previous = 600 
                        print("opening positions according to agent 600")
                        for i in range(self.assets):
                            while True: 
                                try: 
                                    if self.agent600_portfolio[i] > 0: 
                                        self.upbit.buy_market_order(self.tickers[i], cash_amount*(1-self.gamma/100)*self.agent600_portfolio[i])
                                        time.sleep(1)
                                    break  
                                except Exception as e: 
                                    print(e) 
                                    continue                          
                    
                        
                    elif best_gma_idx == 1 and (self.previous != 240 or self.iterations%8==0):   
                        for i in range(self.assets): 
                            while True: 
                                try: 
                                    unit = self.upbit.get_balance(self.tickers[i]) 
                                    self.upbit.sell_market_order(self.tickers[i], unit) 
                                    time.sleep(1)
                                    break 
                                except Exception as e: 
                                    print(e) 
                                    continue 
                        print("storing cash status")
                        cash_amount = self.upbit.get_balance("KRW") 
                        print("current cash amount = {:.2f} won".format(cash_amount)) 
                        self.cash_status.append(cash_amount)
                        self.previous = 240 
                        print("opening positions according to agent 240")
                        for i in range(self.assets):
                            while True: 
                                try: 
                                    if self.agent240_portfolio[i] > 0: 
                                        self.upbit.buy_market_order(self.tickers[i], cash_amount*(1-self.gamma/100)*self.agent240_portfolio[i])
                                        time.sleep(1)
                                    break  
                                except Exception as e: 
                                    print(e) 
                                    continue 


                    elif best_gma_idx == 2 and (self.previous != 60 or self.iterations%2==0): 
                        for i in range(self.assets): 
                            while True: 
                                try: 
                                    unit = self.upbit.get_balance(self.tickers[i]) 
                                    self.upbit.sell_market_order(self.tickers[i], unit) 
                                    time.sleep(1)
                                    break 
                                except Exception as e: 
                                    print(e) 
                                    continue 
                        print("storing cash status")
                        cash_amount = self.upbit.get_balance("KRW") 
                        print("current cash amount = {:.2f} won".format(cash_amount)) 
                        self.cash_status.append(cash_amount)
                        self.previous = 60 
                        print("opening positions according to agent 60")
                        for i in range(self.assets):
                            while True: 
                                try: 
                                    if self.agent60_portfolio[i] > 0: 
                                        self.upbit.buy_market_order(self.tickers[i], cash_amount*(1-self.gamma/100)*self.agent60_portfolio[i])
                                        time.sleep(1)
                                    break  
                                except Exception as e: 
                                    print(e) 
                                    continue 
                            
                    elif best_gma_idx == 3 and (self.previous != 30 or self.iterations%1==0): 
                        for i in range(self.assets): 
                            while True: 
                                try: 
                                    unit = self.upbit.get_balance(self.tickers[i]) 
                                    self.upbit.sell_market_order(self.tickers[i], unit) 
                                    time.sleep(1)
                                    break 
                                except Exception as e: 
                                    print(e) 
                                    continue 
                        print("storing cash status")
                        cash_amount = self.upbit.get_balance("KRW") 
                        print("current cash amount = {:.2f} won".format(cash_amount)) 
                        self.cash_status.append(cash_amount)
                        self.previous = 30 
                        print("opening positions according to agent 30")
                        for i in range(self.assets):
                            while True: 
                                try: 
                                    if self.agent30_portfolio[i] > 0: 
                                        self.upbit.buy_market_order(self.tickers[i], cash_amount*(1-self.gamma/100)*self.agent30_portfolio[i])
                                        time.sleep(1)
                                    break  
                                except Exception as e: 
                                    print(e) 
                                    continue 
                            
                else:
                    print("choosing cash agent as all agents are doing badly") 
                    for i in range(self.assets): 
                        while True: 
                            try: 
                                unit = self.upbit.get_balance(self.tickers[i]) 
                                self.upbit.sell_market_order(self.tickers[i], unit) 
                                time.sleep(1)
                                break 
                            except Exception as e: 
                                print(e) 
                                continue 
                    self.previous = 30 
                    return self.cash_agent_portfolio 
                
            ''' increment iterations '''
            self.iterations += 1 

    def run(self): 
        print("runs every 30 minutes, decides the best agent") 
        while True:
            t0 = time.time() 
            print("Trade {}".format(self.iterations))
            self.trade() 
            if self.previous == 600: 
                print("Chose agent 600")
                print("agent 600 portfolio weights = {}".format(self.agent600_portfolio))
            elif self.previous == 240: 
                print("Chose agent 240") 
                print("agent 240 portfolio weights = {}".format(self.agent240_portfolio)) 
            elif self.previous == 60: 
                print("Chose agent 60") 
                print("agent 60 portfolio weights = {}".format(self.agent60_portfolio)) 
            elif self.previous == 30: 
                print("Chose agent 30 or cash agent") 
                print("agent 30 portfolio weights = {}".format(self.agent30_portfolio)) 
                
            print("Waiting for the next 30 minutes...")
            elapsed = time.time()-t0
            time.sleep(60*30 - elapsed) 
            

In [ ]:
'''
Trade must start at the exact hour 
'''
tickers = ["KRW-BTC", 
           "KRW-ADA", 
           "KRW-XRP", 
           "KRW-DOT", 
           "KRW-DOGE", 
           "KRW-AXS", 
           "KRW-BCH", 
           "KRW-SRM", 
           "KRW-TRX", 
           "KRW-ETH", 
           "KRW-XEC", 
           "KRW-POLY", 
           "KRW-OMG", 
           "KRW-ONG", 
           "KRW-XTZ", 
           "KRW-HIVE", 
           "KRW-LINK", 
           "KRW-XLM", 
           "KRW-EOS", 
           "KRW-ETC"] 
# API keys 
access_key = "ZcG39GRdKaD1jPWS9uk4xZxxWtSAlA92yox1xm5A"
secret_key = "Y4I2SCa0HwGfLKxG2MtuZfiyCKlHaW2M3ljOm7Vs"
upbit = pyupbit.Upbit(access_key, secret_key)

trading_system = Agent_Selector(tickers=tickers, 
                                gma_window_size=20, 
                                wgma_window_size=10, 
                                upbit=upbit, 
                                gamma=0.05, 
                                threshold=0.0) 
trading_system.run() 

  0%|          | 0/20 [00:00<?, ?it/s]

runs every 30 minutes, decides the best agent
Trade 0
current cash amount = 498375.23 won


  5%|▌         | 1/20 [00:00<00:02,  7.32it/s]

First trade agent_600 choosing ONS


  5%|▌         | 1/20 [00:00<00:02,  8.18it/s]

...

agent_30 choosing WMAMR10
agent600 wgma: 0.9992179680433374
agent240 wgma: 1.0004721235448528
agent60 wgma: 0.9972033257840792
agent30 wgma: 1.0033041801067049
agent600 gma: 1.002147388196973
agent240 gma: 0.998365186806463
agent60 gma: 0.9993611336224933
agent30 gma: 1.0009519837672198
choosing agent600
storing cash status
current cash amount = 469997.19 won
opening positions according to agent 600
Chose agent 600
agent 600 portfolio weights = [0.         0.         0.08144209 0.         0.         0.
 0.         0.         0.         0.         0.19451316 0.5785531
 0.         0.         0.         0.14549165 0.         0.
 0.         0.        ]
Waiting for the next 30 minutes...
Trade 28


100%|██████████| 20/20 [00:07<00:00,  2.56it/s]


agent_60 choosing WMAMR5


100%|██████████| 20/20 [00:03<00:00,  6.52it/s]


agent_30 choosing WMAMR10
agent600 wgma: 1.0009163380244839
agent240 wgma: 0.9988338755249663
agent60 wgma: 1.0009254930976532
agent30 wgma: 1.004013467661699
agent600 gma: 1.0018072295828662
agent240 gma: 0.999020235051507
agent60 gma: 0.9994055140305724
agent30 gma: 1.001222341074402
choosing agent600
Chose agent 600
agent 600 portfolio weights = [0.         0.         0.08144209 0.         0.         0.
 0.         0.         0.         0.         0.19451316 0.5785531
 0.         0.         0.         0.14549165 0.         0.
 0.         0.        ]
Waiting for the next 30 minutes...


# Past Runs

In [237]:
'''
Trade must start at the exact hour 
'''
time.sleep(60*19)
tickers = ["KRW-BTC", 
           "KRW-ADA", 
           "KRW-XRP", 
           "KRW-DOT", 
           "KRW-DOGE", 
           "KRW-AXS", 
           "KRW-BCH", 
           "KRW-SRM", 
           "KRW-TRX", 
           "KRW-ETH", 
           "KRW-XEC", 
           "KRW-POLY", 
           "KRW-OMG", 
           "KRW-ONG", 
           "KRW-XTZ", 
           "KRW-HIVE", 
           "KRW-LINK", 
           "KRW-XLM", 
           "KRW-EOS", 
           "KRW-ETC"] 
# API keys 
access_key = "ZcG39GRdKaD1jPWS9uk4xZxxWtSAlA92yox1xm5A"
secret_key = "Y4I2SCa0HwGfLKxG2MtuZfiyCKlHaW2M3ljOm7Vs"
upbit = pyupbit.Upbit(access_key, secret_key)

trading_system = Agent_Selector(tickers=tickers, 
                                gma_window_size=20, 
                                wgma_window_size=10, 
                                upbit=upbit, 
                                gamma=0.05, 
                                threshold=0.0) 
trading_system.run() 

  0%|          | 0/20 [00:00<?, ?it/s]

runs every 30 minutes, decides the best agent
Trade 0
current cash amount = 473234.07 won


  5%|▌         | 1/20 [00:00<00:03,  5.64it/s]

First trade agent_600 choosing ONS


  5%|▌         | 1/20 [00:00<00:02,  7.20it/s]

...

100%|██████████| 20/20 [00:13<00:00,  1.51it/s]


agent_30 choosing WMAMR15
agent600 wgma: 1.0087996484945592
agent240 wgma: 0.9995601664856805
agent60 wgma: 1.0062906327992611
agent30 wgma: 0.9995964656739187
agent600 gma: 1.0026886413455824
agent240 gma: 1.0009402425859353
agent60 gma: 0.9985276445534947
agent30 gma: 0.995714845188139
choosing agent600
storing cash status
current cash amount = 498375.23 won
opening positions according to agent 600
Chose agent 600
agent 600 portfolio weights = [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan]
Waiting for the next 30 minutes...
Trade 21


100%|██████████| 20/20 [00:08<00:00,  2.47it/s]


agent_30 choosing WMAMR15
agent600 wgma: nan
agent240 wgma: 0.9998905259638107
agent60 wgma: 1.005062316688137
agent30 wgma: 0.996332656827
Chose agent 600
agent 600 portfolio weights = [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan]
Waiting for the next 30 minutes...


KeyboardInterrupt: 